# 根据给定的训练集 生成sft格式的数据集用于大模型微调
# sft数据集格式：
jsonl文件
每一行：{"instruction": "请说一下化学反应动力学的应用现状。", "input": "", "output": "化学反应动力学的应用现状包括废水处理、生产过程的反应条件优化、催化剂的设计和评价等领域。例如，化学反应动力学可以用于确定反应机理和催化剂的性能，以更好地解决重要的基础和工业问题。"}

# 训练集格式：
{
        "text": "As observed for the glycosylations of the 4-OH acceptor 7 , the use of equimolar proportions of the donor 6 and the acceptor 16 , in glycosylation reactions , led to the isolation of significant quantities of the glucal 9 and low yields of the disaccharides.",
        "metainfo": "#\tpassage=10.1021/ja00144a001-8\tsegment=1",
        "reactions": [
            {
                "Yield": [
                    "significant quantities"
                ],
                "Product": [
                    "9"
                ]
            },
            {
                "Yield": [
                    "low"
                ],
                "Product": [
                    "disaccharides."
                ]
            }
        ]
    }

In [1]:
import os 
import json
from data_process import *
from prompt_construct import *
base_dir=os.getcwd()
data_dir=base_dir+"/data/train.json"

In [2]:
#读数据
train_set=read_json_file(data_dir)
res_list=[]
for sentence_item in train_set:
    sentence=sentence_item['text']
    reactions=sentence_item['reactions']
    for reaction_item in reactions:
        sft_dict={}
        product=reaction_item['Product']
        instruction,input=gen_sft_template(sentence,product)
        output=json.dumps(reaction_item)
        sft_dict['instruction']=instruction
        sft_dict['input']=input
        sft_dict['output']=output
        res_list.append(sft_dict)

save_dir=base_dir+'/reaction_sft.json'
write_dict_to_file(res_list,save_dir)

In [3]:
data_dir=base_dir+'/reaction_sft.json'
data=read_json_file(data_dir)
print(len(data))

599


# 生成product类实体的sft数据集

In [1]:
import os 
import json
from data_process import *
from prompt_construct import *
base_dir=os.getcwd()
data_dir="/data/lizh/ChemRxnExtractor-main/llm_role/data/prod/train.json"
#读数据
train_set=read_json_file(data_dir)
res_list=[]

for data_item in train_set:
    sentence=data_item['text']
    
    sft_dict={}
    instruction,input=gen_product_sft(sentence)
    entities=data_item['products']

    sft_dict['instruction']=instruction
    sft_dict['input']=input
    sft_dict['output']=str(entities)
    res_list.append(sft_dict)

save_dir=base_dir+'/product_sft.json'
write_dict_to_file(res_list,save_dir)